## Vector Database (VectorDB) and Vector I/O (VectorIO)

Getting Started with VectorDB and VectorIO APIs Tutorial 🚀
Welcome! This interactive tutorial will guide you through using the VectorDB and VectorIO APIs, powerful tools for document storage and retrieval. Whether you're new to vector databases or an experienced developer, this notebook will help you understand the basics and get up and running quickly.
What you'll learn:

How to set up and configure the VectorDB and VectorIO client
Creating and managing vector databases
Different ways to insert documents into the system
How to perform intelligent queries on your documents

Prerequisites:

Basic Python knowledge
A running instance of the Llama Stack server (we'll use localhost in 
this tutorial)

Before you begin, please ensure Llama Stack is installed and set up by following the [Getting Started Guide](https://llama-stack.readthedocs.io/en/latest/getting_started/index.html).

Let's start by installing the required packages:

Set up your connection parameters:

In [ ]:
HOST = "localhost"  # Replace with your host
PORT = 8321        # Replace with your port
MODEL_NAME='meta-llama/Llama-3.2-3B-Instruct'
VECTOR_DB_ID="tutorial_db"

In [ ]:
# Install the client library and a helper package for colored output
#!pip install llama-stack-client termcolor

# 💡 Note: If you're running this in a new environment, you might need to restart
# your kernel after installation

1. **Initial Setup**

First, we'll import the necessary libraries and set up some helper functions. Let's break down what each import does:

llama_stack_client: Our main interface to the VectorDB and VectorIO APIs
base64: Helps us encode files for transmission
mimetypes: Determines file types automatically
termcolor: Makes our output prettier with colors

❓ Question: Why do we need to convert files to data URLs?
Answer: Data URLs allow us to embed file contents directly in our requests, making it easier to transmit files to the API without needing separate file uploads.

In [ ]:
import base64
import json
import mimetypes
import os
import requests
from pathlib import Path

from llama_stack_client import LlamaStackClient
from llama_stack_client.types import Document
from llama_stack_client.types.vector_io_insert_params import Chunk
from termcolor import cprint

# Helper function to convert files to data URLs
def data_url_from_file(file_path: str) -> str:
    """Convert a file to a data URL for API transmission

    Args:
        file_path (str): Path to the file to convert

    Returns:
        str: Data URL containing the file's contents

    Example:
        >>> url = data_url_from_file('example.txt')
        >>> print(url[:30])  # Preview the start of the URL
        'data:text/plain;base64,SGVsbG8='
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    with open(file_path, "rb") as file:
        file_content = file.read()

    base64_content = base64.b64encode(file_content).decode("utf-8")
    mime_type, _ = mimetypes.guess_type(file_path)

    data_url = f"data:{mime_type};base64,{base64_content}"
    return data_url

# Helper function to download content from URLs
def download_from_url(url: str) -> str:
    """Download content from a URL

    Args:
        url (str): URL to download content from

    Returns:
        str: Content of the URL
    """
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Failed to download content from {url}: {response.status_code}")

2. **Initialize Client and Create Vector Database**

Now we'll set up our connection to the VectorDB API and create our first vector database. A vector database is a specialized database that stores document embeddings for semantic search.
❓ Key Concepts:

embedding_model: The model used to convert text into vector representations
chunk_size: How large each piece of text should be when splitting documents
overlap_size: How much overlap between chunks (helps maintain context)

✨ Pro Tip: Choose your chunk size based on your use case. Smaller chunks (256-512 tokens) are better for precise retrieval, while larger chunks (1024+ tokens) maintain more context.

In [ ]:
# Initialize client
client = LlamaStackClient(
    base_url=f"http://{HOST}:{PORT}",
)

# Let's see what providers are available
# Providers determine where and how your data is stored
providers = client.providers.list()
vector_io_providers = [p for p in providers if p.api == "vector_io"]
provider_id = vector_io_providers[0].provider_id if vector_io_providers else None
print("Available providers:")
print(providers)

# Create a vector database with optimized settings for general use
client.vector_dbs.register(
    vector_db_id=VECTOR_DB_ID,
    embedding_model="all-MiniLM-L6-v2",
    embedding_dimension=384,  # This is the dimension for all-MiniLM-L6-v2
    provider_id=provider_id,
)

3. **Insert Documents**
   
The VectorIO API supports multiple ways to add documents. We'll demonstrate two common approaches:

Loading documents from URLs
Loading documents from local files

❓ Important Concepts:

Each document needs a unique document_id
Metadata helps organize and filter documents later
The API automatically processes and chunks documents

In [ ]:
# Example URLs to documentation
# 💡 Replace these with your own URLs or use the examples
urls = [
    "memory_optimizations.rst",
    "chat.rst",
    "llama3.rst",
]

# Create documents from URLs
# We add metadata to help organize our documents
url_documents = []
for i, url in enumerate(urls):
    full_url = f"https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/{url}"
    try:
        # Download content from URL
        content = download_from_url(full_url)
        # Create document with the downloaded content
        document = Document(
            document_id=f"url-doc-{i}",  # Unique ID for each document
            content=content,  # Use the actual content instead of the URL
            mime_type="text/plain",
            metadata={"source": "url", "filename": url, "original_url": full_url},  # Store original URL in metadata
        )
        url_documents.append(document)
        print(f"Successfully downloaded content from {url}")
    except Exception as e:
        print(f"Failed to download content from {url}: {e}")

# Example with local files
# 💡 Replace these with your actual files
local_files = ["example.txt", "readme.md"]
file_documents = []
for i, path in enumerate(local_files):
    if os.path.exists(path):
        try:
            # Read content from file directly instead of using data URL
            with open(path, 'r') as file:
                content = file.read()
            document = Document(
                document_id=f"file-doc-{i}",
                content=content,  # Use the actual content directly
                mime_type="text/plain",
                metadata={"source": "local", "filename": path},
            )
            file_documents.append(document)
            print(f"Successfully read content from {path}")
        except Exception as e:
            print(f"Failed to read content from {path}: {e}")

# Combine all documents
all_documents = url_documents + file_documents

# Create chunks from the documents
chunks = []
for doc in all_documents:
    # Split document content into chunks of 512 characters
    content = doc.content
    chunk_size = 512
    
    # Create chunks of the specified size
    for i in range(0, len(content), chunk_size):
        chunk_content = content[i:i+chunk_size]
        if chunk_content.strip():  # Only add non-empty chunks
            chunks.append(Chunk(
                content=chunk_content,
                metadata={
                    "document_id": doc.document_id,
                    "chunk_index": i // chunk_size,
                    **doc.metadata
                }
            ))

# Insert chunks into vector database
if chunks:  # Only proceed if we have valid chunks
    client.vector_io.insert(
        vector_db_id=VECTOR_DB_ID,
        chunks=chunks,
    )
    print(f"Documents inserted successfully! ({len(chunks)} chunks)")
else:
    print("No valid documents to insert.")

4. **Query the Vector Database**
   
Now for the exciting part - querying our documents! The VectorIO API uses semantic search to find relevant content based on meaning, not just keywords.
❓ Understanding Scores:

Generally, scores above 0.7 indicate strong relevance
Consider your use case when deciding on score thresholds

In [ ]:
def print_query_results(query: str):
    """Helper function to print query results in a readable format

    Args:
        query (str): The search query to execute
    """
    print(f"\nQuery: {query}")
    print("-" * 50)
    response = client.vector_io.query(
        vector_db_id=VECTOR_DB_ID,
        query=query,
    )

    for i, (chunk, score) in enumerate(zip(response.chunks, response.scores)):
        print(f"\nResult {i+1} (Score: {score:.3f})")
        print("=" * 40)
        print(chunk.content)
        print("=" * 40)

# Let's try some example queries
queries = [
    "How do I use LoRA?",  # Technical question
    "Tell me about memory optimizations",  # General topic
    "What are the key features of Llama 3?"  # Product-specific
]


for query in queries:
    print_query_results(query)

Awesome, now we can embed all our notes with Llama-stack using VectorDB and VectorIO, and ask it about the meaning of life :)

Next up, we will learn about the safety features and how to use them: [notebook link](./06_Safety101.ipynb).